In [1]:
using JuMP
using CSV
using LinearAlgebra
using DataFrames
using BARON

In [3]:
A_db = CSV.File("./A_mat_new.csv",header=1,delim=",") |> DataFrame
B_db = CSV.File("./B_mat_new.csv",header=1,delim=",") |> DataFrame
f_db = CSV.File("./f_val_new.csv",header=1,delim=",") |> DataFrame
f_val=f_db[!,2]
A_mat=Matrix(A_db[:,2:size(A_db,2)])
B_mat=Matrix(B_db[:,2:size(B_db,2)])
nodes_val=names(A_db)[2:size(A_db,2)]
product_val=A_db[!,1]

30-element Vector{String63}:
 "Methanol "
 "Ethylene "
 "Propylene "
 "Toluene "
 "Benzene "
 "Xylene "
 "Butene "
 "Propane "
 "Syngas"
 "Urea "
 "Acrylonitrile"
 "Cyclohexane "
 "Caprolactam "
 ⋮
 "Vinyl chloride Monomer "
 "Polypropylene "
 "Styrene "
 "Ethylene glycol "
 "Terephthalic Acid (PTA) "
 "Polyethylene Terephthalate (PET)"
 "Propylene Oxide "
 "Butadiene "
 "Acetic Acid "
 "Formaldehyde "
 "Adipic Acid"
 "Gasoline Additives"

In [4]:
B_db

,Column1,Syngas Production,Methanol Manufacture,Steam Cracker,Hydrodealkylation (THD)
,String31,Float64,Float64,Float64,Float64
1,Coal,-55.6882,-25.4601,0.0,0.0
2,Natural Gas,-52.8947,-24.0426,-62.8951,0.0
3,Waste,20.7175,11.981,15.6427,0.251672
4,Environment,31.7886,13.2328,0.0,0.942308
5,Oil,0.0,-6.29662,-241.801,0.0
6,Limestone,0.0,0.0,0.0,0.0
7,Coal Energy,-59.4291,-18.6662,-87.5141,-0.584625
8,NG Energy,-31.7968,-9.98711,0.0,-0.312796
9,Environment Energy,91.2258,28.6533,87.5141,0.897421


In [15]:
function opt_decarb()
    dcrb =  Model(BARON.Optimizer)
    @variable(dcrb,s[j=1:size(A_mat,2)] ≥ 0)
    @variable(dcrb,f[i=1:size(A_mat,1)] ≥ 0 )
    @variable(dcrb,g[k=1:size(B_mat,1)])
    @constraint(dcrb,[i=1:size(A_mat,1)],f[i] ≥ f_val[i]/2)
    @constraint(dcrb,[i=1:size(A_mat,1)], f[i]==sum(A_mat[i,j]*s[j] for j in 1:size(A_mat,2)))
    @constraint(dcrb,[k=1:size(B_mat,1)],g[k]==sum(B_mat[k,j]*s[j] for j in 1:size(B_mat,2)))
    @variable(dcrb,obj)
    @constraint(dcrb,obj==sum(g[k] for k in 3:4)+g[9])
    @objective(dcrb,Min,obj)
    optimize!(dcrb)
    return JuMP.value.(s),JuMP.value.(f),JuMP.value.(obj)
end
    

opt_decarb (generic function with 1 method)

In [16]:
s,f,obj=opt_decarb()

 BARON version 21.1.13. Built: LNX-64 Wed Jan 13 16:09:31 EST 2021

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Bhavik R. Bakshi at The Ohio State University, bakshi.2@osu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Solving bounding LP
 Preprocessing found feasible solution with value  335.774356936     
 Problem solved during preprocessing
 Lower bound is  335.774356936     

 Calculating duals

                         *** Normal completion ***            

 Wall clock time:                     0.01
 Total CPU time used:             

([0.4999999999643346, 0.5, 0.5045278323344176, 0.46810720914193826, 0.5347892315026733, 0.9790446607729414, 0.0, 0.9790446607729415, 0.5, 0.5  …  0.5000000000000001, 0.4999999999999999, 0.5, 0.5, 0.5, 0.5000000000000001, 0.5, 0.5, 0.5000000000000001, 0.5], [12.945509024, 20.127333526, 0.377063157500004, 21.4456166735, 5.2862515295, 6.66562665054647, 0.0, 6.01161903004908, 0.0, 21.8  …  7.33846154, 3.2601290315, 4.667566265, 20.965641665, 3.4758620690000006, 9.982769123822573, 3.6, 4.173, 0.9616438355000001, 104.55992055], 335.77435693558664)

In [17]:
obj

335.77435693558664

In [18]:
res_f=DataFrame(pdt=product_val, final_demand=f)
res_s=DataFrame(node=nodes_val, scaling_factor=s)

,node,scaling_factor
,String,Float64
1,Syngas Production,0.5
2,Methanol Manufacture,0.5
3,Steam Cracker,0.504528
4,Hydrodealkylation (THD),0.468107
5,Disproportionation (TDP),0.534789
6,Metathesis,0.979045
7,Dehydrogenation,0.0
8,Refinery,0.979045
9,Urea Synthesis,0.5


In [19]:
res_f

,pdt,final_demand
,String63,Float64
1,Methanol,12.9455
2,Ethylene,20.1273
3,Propylene,0.377063
4,Toluene,21.4456
5,Benzene,5.28625
6,Xylene,6.66563
7,Butene,0.0
8,Propane,6.01162
9,Syngas,0.0


In [20]:
B_db

,Column1,Syngas Production,Methanol Manufacture,Steam Cracker,Hydrodealkylation (THD)
,String31,Float64,Float64,Float64,Float64
1,Coal,-55.6882,-25.4601,0.0,0.0
2,Natural Gas,-52.8947,-24.0426,-62.8951,0.0
3,Waste,20.7175,11.981,15.6427,0.251672
4,Environment,31.7886,13.2328,0.0,0.942308
5,Oil,0.0,-6.29662,-241.801,0.0
6,Limestone,0.0,0.0,0.0,0.0
7,Coal Energy,-59.4291,-18.6662,-87.5141,-0.584625
8,NG Energy,-31.7968,-9.98711,0.0,-0.312796
9,Environment Energy,91.2258,28.6533,87.5141,0.897421


In [21]:
f_db

,Product,Final Demands
,String63,Float64
1,Methanol,25.891
2,Ethylene,40.2547
3,Propylene,0.754126
4,Toluene,42.8912
5,Benzene,10.5725
6,Xylene,12.9475
7,Butene,0.0
8,Propane,0.0
9,Syngas,4.0e-9


## Sensitivity optimization

In [23]:
function sens_opt_decarb()
    dcrb =  Model(BARON.Optimizer)
    @variable(dcrb,s[j=1:size(A_mat,2)] ≥ 0)
    @variable(dcrb,f[i=1:size(A_mat,1)] ≥ 0 )
    @variable(dcrb,g[k=1:size(B_mat,1)])
    
    
    @variable(dcrb,y1[i=1:size(A_mat,1),j=1:size(A_mat,2)],Bin)
    @variable(dcrb,y2[i=1:size(A_mat,1),j=1:size(A_mat,2)],Bin)
    @variable(dcrb,y[i=1:size(A_mat,1),j=1:size(A_mat,2)],Bin)
    @constraint(dcrb,[i=1:size(A_mat,1),j=1:size(A_mat,2)],y[i,j] == y1[i,j]+y2[i,j])
    @constraint(dcrb,sum(y[i,j] for i in 1:size(A_mat,1), j=1:size(A_mat,2)) ≤ 10) 
    
    @constraint(dcrb,[i=1:size(A_mat,1)],f[i] ≥ f_val[i]/2)
    @constraint(dcrb,[i=1:size(A_mat,1)], f[i]==sum(A_mat[i,j]*(1+0.1*y1[i,j]-0.1*y2[i,j])*s[j] for j in 1:size(A_mat,2)))
    @constraint(dcrb,[k=1:size(B_mat,1)],g[k]==sum(B_mat[k,j]*s[j] for j in 1:size(B_mat,2)))
    @variable(dcrb,obj)
    @constraint(dcrb,obj==sum(g[k] for k in 3:4)+g[9])
    @objective(dcrb,Min,obj)
    optimize!(dcrb)
    return JuMP.value.(s),JuMP.value.(f),JuMP.value.(obj),JuMP.value.(y1),JuMP.value.(y2)
end
    

sens_opt_decarb (generic function with 1 method)

In [24]:
s,f,obj,y1,y2=sens_opt_decarb()

 BARON version 21.1.13. Built: LNX-64 Wed Jan 13 16:09:31 EST 2021

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Bhavik R. Bakshi at The Ohio State University, bakshi.2@osu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Kilinc, M. and N. V. Sahinidis, Exploiting integrality in the global
 optimization of mixed-integer nonlinear programming problems in BARON,
 Optimization Methods and Software, 33, 540-562, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Doing local search
 Preprocessing found feasible solution with value  298.679822361     
 Solving bounding LP
 Starting multi-start local search
 Done with local search
  Iteration    Open nodes         Time (s)    Lower bound      Upper bound
*         1             1             1.79

([0.37630081447175906, 0.4439920788770053, 0.45566289410676575, 0.8030579939583976, 0.17328578352411647, 1.5289611146171238, 0.0, 1.5289611146171238, 0.4545454545454546, 0.5  …  0.5, 0.4298746490543581, 0.4637170454546182, 0.45454545454545453, 0.5, 0.5, 0.5, 0.49999999999999994, 0.5, 0.500964349303155], [12.945509024, 20.127333526, 0.377063157500004, 21.4456166735, 5.2862515295, 6.4737715665, 0.0, 9.388266032298032, 0.0, 21.8  …  7.33846154, 3.2601290315, 4.667566265, 20.965641665, 3.475862069, 9.9175008721028, 3.6, 4.173, 0.9616438355, 104.76158512304069], 297.76247104389023, [0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [25]:
print(" Positive perturbations \n\n")

for i in 1:size(findall(x->x==1, y1),1)
    print(nodes_val[findall(x->x==1, y1)[i][2]]*" ---- "*product_val[findall(x->x==1, y1)[i][1]]*"\n")
end   
print("\n Negative perturbations \n\n")
for i in 1:size(findall(x->x==1, y2),1)
    print(nodes_val[findall(x->x==1, y2)[i][2]]*" ---- "*product_val[findall(x->x==1, y2)[i][1]]*"\n")
end   

 Positive perturbations 

Syngas Production ---- Syngas
Methanol Manufacture ---- Methanol 
Steam Cracker ---- Ethylene 
Steam Cracker ---- Butadiene 
Urea Synthesis ---- Urea 
EG Production ---- Ethylene glycol 
PET Production ---- Polyethylene Terephthalate (PET)

 Negative perturbations 

Methanol Manufacture ---- Syngas
Formaldehyde production ---- Methanol 
